# Versuch V-So 15 – Maschinelles Lernen in der wissenschaftlichen Bildanalyse

# Einleitung

Ziel dieses Versuchs ist es physikalische Eigenschaften
- *Position*
- *lokale Dichte*
- *Volumen*
- *Intensität*
- usw.

von Zellen mit Deep Learning aus Mikroskopaufnahmen zu extrahieren. Der Versuch ist im wesentlich in vier Phasen geteilt:
1. Einführung
2. Vorbereitung der Trainingsdaten
3. Erstellen von geeigneten Netzwerken
4. Visualisierung der Daten

# Hinweise

Dies ist keine Prüfungssituation!
- Zu jedem Zeitpunkt kann eine [Suchmaschine](https://en.wikipedia.org/wiki/Web_search_engine) euer Vorankommen beschleunigen.
- Wenn ein Problem auftritt, dann sucht nach Beispiele und Lösungen auf [Github](https://github.com/), [StackOverflow](https://stackoverflow.com/) und in offiziellen Dokumentationen (z.B. [Matplotlib](https://matplotlib.org/gallery/index.html)).
- Es erwartet keiner, dass ihr alle Python [Befehle](https://docs.python.org/3/library/) auswendig kennt. Es wird aber erwartet, dass ihr in kurzer Zeit euch neue Befehle heraussuchen könnt.
- Wenn ihr Fragen habt, lasst es uns wissen.
- Speichert das Notebook, immer wenn ihr es für notwendig haltet.

# 1. Einführung

## Macht euch mit der Maschine vertraut:

Die für die Numerik sind folgende Komponenten von Bedeutung:
- CPU (Model)
- Arbeitsspeicher (Speicherplatz)
- Speicher (Model (von nvme0n1))
- GPU (Model)

Ohne diese Hardwarekomponenten könnte der Versuch nicht durchgeführt werden. Sie sollten deshalb im Protokoll Erwähnungen finden. Mit den folgenden Befehlen könnt ihr sie ganz einfach auslesen:

In [ ]:
!cat /proc/cpuinfo | \
awk -v FS=':' '                                       \
  /^physical id/ { if(nb_cpu<$2)  { nb_cpu=$2 } }     \
  /^cpu cores/   { if(nb_cores<$2){ nb_cores=$2 } }   \
  /^processor/   { if(nb_units<$2){ nb_units=$2 } }   \
  /^model name/  { model=$2 }                         \
                                                      \
  END{                                                \
   nb_cpu=(nb_cpu+1);                                 \
   nb_units=(nb_units+1);                             \
                                                      \
   print "CPU model:",model;                          \
   print nb_cpu,"CPU,",nb_cores,"physical cores per CPU, total",nb_units,"logical CPU units" \
 }' # Quelle: https://superuser.com/questions/388115/interpreting-output-of-cat-proc-cpuinfo

In [ ]:
!cat /proc/meminfo | grep MemTotal

In [ ]:
!cat /sys/class/block/nvme0n1/device/model

In [ ]:
!nvidia-smi --query-gpu=gpu_name,memory.total --format=csv

## Macht euch mit der Daten vertraut

Im Gegensatz zu vielen anderen Versuche im Rahmen des Fortgeschrittenenpraktikums, ist ein Großteil der für diesen Versuch notwendigen Daten bereits vorhanden. Ihr findet die Daten unter:


**Traininginput** - Mikroskopaufnahmen.

`/DATA/IN/<Experiment Name>/`

**Traininglabels (Ground Truth)** - Von den Mikroskopaufnahmen erstellte Binärmasken.

`/DATA/GT/<Experiment Name>/`

**Anwendungsdatensätze** - Datensätze für eure Analyse.

`/DATA/validation/noise1000/`

**Notebook & Benutzerdaten** - Speicherort für *.ipynb* und allen Daten die ihr im Laufe des Versuchs generiert.

`/tf/Notebooks`

#### Aufgabe 1:
- Benutzt das [Kommandozeilenprogramm](https://linux.die.net/man/1/ls) `ls` um euch mit dem Inhalt der Verzeichnisse vertraut zu machen.
- <font id="overview">Nutzt</font> die unten definierten Funktionen `loadImage` und `zSlicer` um  2 verschiedenen Trainingslabels und -Inputpaare (also ingesamt 4 Diagramme) anzuzeigen.

Hinweise:

- Mit vorangestellten `!` könnt ihr in diesem Notebook beliebige Programme auf dem Computer ausführen.

In [ ]:
%matplotlib notebook

import numpy as np
import imread
import ipywidgets as widgets
import matplotlib.pyplot as plt

def loadImage(filepath):
    """
    Liest tiff-Dateien von der Festplatte als numpy array in den Arbeitsspeicher.
    
    Parameter
    ----------
    filepath : string
        Vollständiger Pfad der tiff-Datei.
        
    
    Rückgabewert
    ------------
    out : array_like
        Tiff-Datei als 3D numpy Array.
    """    
    
    return np.asarray(imread.imload_multi(filepath))

def zSlicer(*args):
    """
    Zeichnet ein interactives Diagramm einer beliebigen Anzahl an 3D oder 2D numpy Arrays.
    
    Über einen Slider unter dem Diagramm kann die dargestellte z-Ebene verändert werden.
    
    Parameter
    ----------
    *args : array_like
        Ein (oder mehrere komma-getrennte) 2D bzw. 3D numpy Array(s)
    """
    
    num_img = len(args)
            
    num_rows = 1
    num_cols = len(args)
    
    # init plot
    fig, axes = plt.subplots(num_rows, num_cols, figsize=[9.5,5])
    
    val = 0
    aspect = 'equal'
    
    plots = []
    for i, img in enumerate(args):
        if len(args) == 1:
            ax = axes
        else:
            ax = axes[i]
        
        if img.ndim > 2:

            plots.append([ax.imshow(img[val], aspect=aspect, vmin=np.amin(img[1:]), vmax=np.amax(img[1:])), img])

        else:
            ax.imshow(img, aspect=aspect, vmin=np.amin(img), vmax=np.amax(img))
            

    def update_plot(change):
        val = change['new']
        for plot,img in plots:
            plot.set_array(img[val])
            
        return


    slider = widgets.IntSlider(value=val, max=args[-1].shape[0]-1)
    display(slider)

    slider.observe(update_plot, names='value')

In [ ]:
#### Lösung

### Limitiere CUDA auf eine GPU

Im Verlauf dieses Versuchs werden Grafikkarten benutzt um Tensoroperationen durchzuführen. Dies soll mithilfe des [*tensorflow*](https://www.tensorflow.org/) Python Moduls passieren. Standardmäßig nutzt *tensorflow*  fast den gesamten verfügbare Grafikspeicher.

Die momentane Auslastung der Grafikkarten auf den System lassen sich mit dem Programm *nvidia-smi* überprüfen.

In [ ]:
!nvidia-smi

Über die [Umgebungsvariable](https://de.wikipedia.org/wiki/Umgebungsvariable) *CUDA_VISIBLE_DEVICES* kann man die Geräte einschränken, die für [CUDA](https://de.wikipedia.org/wiki/CUDA) Berechnungen benutzt werden dürfen.

`%env` ist *jupyter* [*Magie*](https://ipython.readthedocs.io/en/stable/interactive/magics.html) um Umgebungsvariablen zu setzen oder anzeigen zu lassen.

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
# Limit GPU usage to actually needed memory #https://kobkrit.com/using-allow-growth-memory-option-in-tensorflow-and-keras-dc8c8081bc96
from tensorflow.keras.backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.25

sess = tf.Session(config=config)

set_session(sess)

## Keras Einführung

[Keras](https://keras.io/) ist eine high-level Deep Learning Bibliothek, die es erlaubt eigene Designs für neuronale Netzwerke zu basteln, ohne das *TensorFlow* oder gar *CUDA* Code geschrieben werden muss. Dabei kann Keras mit einer Vielzahl von Deep Learning Backends benutzt werden. Wir werden es ausschließlich mit dem Tensorflow Backend benutzen.

Im Laufe dieses Versuchs wollen wir Schritt für Schritt einen [Autoencoder](http://science.sciencemag.org/content/313/5786/504/tab-pdf) erstellen, der dem Design von [UNet](http://arxiv.org/abs/1505.04597) entspricht.

### Die Keras Model API

Keras bezeichnet jedes Netzwerk unabhängig von der Architektur als [*Modell*](https://keras.io/models/model/).
Die Modell API von Keras kommen einr [Reihe](https://keras.io/models/model/) von nützlichen Methoden, die uns [Kompilieren](https://keras.io/getting-started/sequential-model-guide/#compilation), [Training](https://keras.io/getting-started/sequential-model-guide/#training), und [Vorhersagen](https://keras.io/models/model/#predict) mit dem Netzwerk erleichtern.

#### Beispiel: Erstellen eines Modells

Ein Modell wird erstellt indem man die von Keras in `tensorflow.keras.layers` zur Verfügung gestellten Schichten ineinander schachtelt. Dabei wird jeder Schicht (mit Ausnahme der Inputschicht) die vorherige Schicht als Argument übergeben.

Im folgenden Beispiel definieren wir uns zunächst eine (Python-) Funktion, die für uns ein sehr einfaches Convolutionary Neural Network (CNN) erstellt. Dabei wird drei Mal hintereinander eine 2D [Konvolution](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1) auf einen Input der Größe $64 \times 64 \times 1$ (z.B. ein Graustufenbild mit $64 \times 64$ Pixel) angewendet.

$$f^{(1)}: \mathbb{R}^{64\times 64 \times 1} \longmapsto \mathbb{R}^{64\times 64\times 1}\\ \pmb{x} \longrightarrow \pmb{x} \ast \pmb{k}_1\\\\ f^{(2)}: \mathbb{R}^{64\times 64 \times 1} \longmapsto \mathbb{R}^{64\times 64\times 1}\\ \pmb{x} \longrightarrow \pmb{x} \ast \pmb{k}_2\\\\ f^{(3)}: \mathbb{R}^{64\times 64 \times 1} \longmapsto \mathbb{R}^{64\times 64\times 1}\\ \pmb{x} \longrightarrow \pmb{x} \ast \pmb{k}_3 \tag{2}\label{eq:simple_model_functions}$$
Das Netzwerk $N$ ist die Verknüpfung dieser Funktionen:
$$N(\pmb{x}) = (f^{(3)} \circ f^{(2)} \circ f^{(1)})(\pmb{x}) \tag{3} \label{eq:simple_model}\\ =  f^{(3)}(f^{(2)}(f^{(1)}(\pmb{x})))$$

In [ ]:
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model

def simple_model(input_shape=(64, 64, 1)):

    # Eingabeschicht des Models; definiert mit welchen Größe von Elementen das Netzwerk arbeiten kann.
    inputs = Input(input_shape)
    
    # Erste Schicht; bekommt die Eingabeschicht als Argument und die Funktionsdefinition als Paramameter übergeben.
    f1 = Conv2D(1, 2, padding='same', activation='relu')(inputs)
    
    # Zweite (verdeckte) Schicht; vgl. erste Schicht.
    f2 = Conv2D(1, 2, padding='same', activation='relu')(f1)
    
    # Dritte (Ausgabe-)Schicht; vfl. erste Schicht.
    f3 = Conv2D(1, 2, padding='same', activation='relu')(f2)

    # Für die Initalisierung des Models wird die Eingabe- und Ausgabeschicht explizit angegeben.
    return Model(inputs=inputs, outputs=f3)

Bei jedem Aufruf der Funktion `simple_model` bekommen wir als Rückgabewert ein fertig initalisiertes `Model`-Objekt geliefert. Die Methoden `summary` von dem Objekt `simple_model` können wir z.B. nutzen um uns eine Übersicht über die Netzwerkarchitektur anzeigen zu lassen.

In [ ]:
# Nutzt die soeben definierte Funktion um ein neues Modell zu initialisieren.
simple_model = simple_model()

# Die Funktion `summary()` ist Teil der Model-API und gibt uns eine Übersicht für das Modell aus.
simple_model.summary()

#### Beispiel: Kompilieren eines Modells

Diese Modell existiert zur Zeit nur als Python Objekt im Arbeitsspeicher. Um die volle Leistungsfähigkeit der Hardware auszunutzen, müssen wir das Python Objekt in Maschinensprache übersetzen (sprich: kompilieren). Dieser Prozess wird uns komplett von der Methode `compile` abgenommen.

In [ ]:
from tensorflow.keras.optimizers import Adam

simple_model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

Die Methode benötigt bereits bei der Kompilierung die Angabe zu der Optimierungsfunktion (hier: [Adam](https://arxiv.org/abs/1412.6980v8) mit einer Anpassungsrate von 0.0001) und zu der Verlustfunktion (hier: [Binäre Kreuz-Entropie](https://de.wikipedia.org/wiki/Kreuzentropie)). Zusätzlich kann man eine Metrik angeben, welche die Leistungsfähigkeit des Modells während des Trainings und Testens misst. 

#### Beispiel: Trainieren eines Modells

Zum Training des Modells benötigen wir Trainingsdaten. In dem folgenden Versuch wollen wir Mikroskopbilder und die zugehörigen Binärmasken von Zellen als Trainingsdaten benutzen. In diesem Beispiel reduzieren wir uns auf Zufallsdaten. Dabei werden insgesamt 1000 Datenpaare für das Training erzeugt (vgl. erste Dimension im folgende Beispiel). `X` stellt die Sammlung an Eingabedaten und `Y` die Sammlung der Labeldaten dar. Die weiteren Dimensionen sind durch die Breite des Netzwerks (bzw. Bilddimensionen der Inputdaten) vorgebeben. Wenn man Bilder als Eingabedaten benutzt stellt die letzte Dimension die Anzahl der Kanäle dar. Bei einem [Graustufenbild](https://www.google.com/search?q=Graustufenbild&source=lnms&tbm=isch) ist dies 1 bei einem [RGB](https://de.wikipedia.org/wiki/RGB-Farbraum) Bild ist die Anzahl 3.

In [ ]:
X, Y = np.random.random((1000, 64, 64, 1)), np.random.random((1000, 64, 64, 1))

In [ ]:
history = simple_model.fit(x=X, y=Y, epochs=10, validation_split=0.1)

#### Beispiel: Visualisieren eines Trainingsprozesses

Die Methode `fit` erzeugt einen Rückgabewert, den wir im vorherigen <a href="#Beispiel:-Trainieren-eines-Modells">Beispiel</a> in der Variable `history` gespeichert haben. `history` enthält für jede Epoche den Trainingsverlauf der Trainingsparameter (hier: *loss* und *acc*). Diese sind für die spätere Evaluation der Konvergenz sehr nützlich und können zu anschauliche Diagramme aufbereitet werden.

In [ ]:
# Data preparation
modelname = 'Beispielnetzwerk'

parameter = history.history.keys() 
YLabels = history.history.keys() 

# Plotting
f, axes = plt.subplots(1, len(parameter), figsize=(4.74*len(parameter), 5))

for ax, param, ylabel in zip(axes, parameter, YLabels):
    y = history.history[param]
    x = np.arange(1, len(y)+1)
    ax.plot(x, y, 'ro', label=modelname)
    ax.set_xlabel('Epoche')
    ax.set_ylabel(ylabel)
    ax.legend()
    ax.set_title(ylabel)
    ax.grid()

plt.tight_layout()

#### Beispiel: Netzwerk zur Vorhersage benutzen

Auch wenn die Konvergenz von Netzwerken in Abhängigkeit von verschiedensten Parameter ein sehr interessantes eigenes [Forschungsfeld](https://arxiv.org/list/cs.AI/recent) darstellt, sind wir eher an der Anwendung der durch das Netzwerk gelernten Funktion interessiert.

Unser Beispielmodell hat sich im Laufe des Trainingsprozesses nicht verändert. Wir haben ein Netzwerk ohne freie Parameter erstellt. Das Modell entspricht weiterhin der Identität im $\mathbb{R}^{m\times64\times64\times1}$.

In dem folgenden Beispiel wollen wir dennoch dieses Netzwerk benutzen um die "gelernte" Identität auf neu erzeugte Inputdaten anzuwenden. Die Inputdaten sind in diesen Fall durch [Schachbrettmuster](https://stackoverflow.com/a/51715491) mit einem zufälligen Skalierungsfaktor gegeben.

Mit der Methode `predict` wenden wir das Netzwerk auf die Daten an. Beachtet hierbei, dass die Anzahl der Inputdaten (hier: 5) nicht notwendigerweise die Anzahl der Trainingspaare (hier: 1000) sein muss. Wir können  das Netzwerk auf einer beliebigen Anzahl von Vorhersagen gleichtzeitig benutzen. Dabei muss nur beachtet werden, dass das komplette Modell inkl. Trainingsdaten in den [Grafikkartenspeicher](#Macht-euch-mit-der-Maschine-vertraut:) passt.

In [ ]:
# Erstelle ein Schachberttmuster
chess_board = np.indices((64, 64)).sum(axis=0) % 2

# Erzeuge eine zusätzliche Dimension für den Farbkanal
chess_board_reshaped = chess_board.reshape(64, 64, 1)

# Erzeuge einen neues Array für die Eingabedaten
X2 = np.zeros((5, 64, 64, 1))
    
# Fülle das Eingabearray mit Schachbrettern und einen zufälligen Faktor
for i in range(5):
    X2[i] = chess_board_reshaped*np.random.random(1)

# Nutze unser Beispielmodell um die gelernte Funktion auf die Eingabedaten anzuwenden
results = simple_model.predict(X2)

# Nutze die bereits definierte Funktion um die Resultate mit den Eingaben zu Vergleichen
zSlicer(X2[..., 0], results[..., 0])

#### Beispiel: Netzwerk und Trainingsverlauf für die spätere Auswertung/ Anwendung speichern

Wir werden später mehrere Netzwerke mit verschiedenen Anzahlen an Schichten zu trainieren. Unser Ziel wird sein das beste dieser Netzwerke für die Anwendung zu benutzen. Dazu müssen wir die Netzwerke und deren Trainingsverlauf auf der Festplatte speichern.

Über die unten definierten Funktionen `save_trained_model` und `load_trained_model` könnt ihr sehr einfach eure Netzwerke auf der Festplatte archivieren.

`save_trained_model` kann neben dem Netzwerkdesign (als [json](https://de.wikipedia.org/wiki/JavaScript_Object_Notation)) und den gelernten Gewichten (als [hdf5](https://de.wikipedia.org/wiki/Hierarchical_Data_Format)) auch den in `history` gespeicherten Trainingsverlauf (als [csv](https://de.wikipedia.org/wiki/CSV_(Dateiformat))) abspeichern.

`load_trained_model` erstellt aus dem gespeicherten Netzwerkdesign und den gespeicherten Gewichten ein neues, fertig trainiertes Modell.

In [ ]:
from datetime import datetime
import csv
import os
from tensorflow.keras.models import model_from_json

def save_trained_model(history, net, foldername):
    
    folder = os.path.join(foldername, datetime.now().strftime('%Y-%m-%d_%H-%M'))
    
    if not os.path.exists(folder):
        os.makedirs(folder)
        
    hist_pivot = [dict(zip(history.history, col)) for col in zip(*history.history.values())] # https://stackoverflow.com/a/37489316
        
    history_path = os.path.join(folder, 'history.csv')
    
    print('Save history in path "{}"'.format(history_path))
    with open(os.path.join(folder, 'history.csv'), 'w') as f:
        w = csv.DictWriter(f, history.history.keys())
        w.writeheader()
        for row in hist_pivot:
            w.writerow(row)
    
    model_json = net.to_json()
    model_path = os.path.join(folder, "model.json")
    print('Save model in path "{}"'.format(model_path))
    with open(model_path, "w") as json_file:
        json_file.write(model_json)
    
    weights_path = os.path.join(folder, "weights.h5")
    print('Save weights in path "{}"'.format(weights_path))
    net.save_weights(weights_path)
    
    return folder
    
def load_trained_model(foldername):
    
    # load json and create model
    print('Load trained model from "{}"'.format(foldername))
    with open(os.path.join(foldername, 'model.json'), 'r') as json_file:
        loaded_model_json = json_file.read()

    net = model_from_json(loaded_model_json)
    # load weights into new model
    net.load_weights(os.path.join(foldername, 'weights.h5'))
    
    return net



In [ ]:
# Save model, weights, and history
save_folder = save_trained_model(history, simple_model, 'beispielnetzwerk')

# ... do something else ...

# Load model and weights
new_simple_model = load_trained_model(save_folder)

Nun stellt sich die Frage: Wie trainieren wir eigentlich ein Netzwerk auf unseren Datensätzen?

# Trainingsdaten einlesen

Wie bereits <a href="#Macht-euch-mit-der-Daten-vertraut">erwähnt</a>, liegen die Labeldaten unter `/DATA/GT/` und zugehörige Inputdaten unter `/DATA/IN/`. Um Trainingsdatenpaare zu erstellen muss man für jedes einzelne Paar die entsprechenden Dateipfade angeben. Damit die von uns erstellten Netzwerke auch mit unbekannte Daten umgehen können, benötigen wir mehrere Hundert Paare. Nur so enthalten unsere Trainingsdaten eine akzeptable Entropie.  

Weil ein manuelles Abschreiben dieser Dateipfade den Zeitrahmen diese Versuchs sprengen würde, lassen wir uns alle Dateipfade ausgeben, die einem bestimmten Muster entsprechen. Dazu benutzen wir die `glob`-Funktion des gleichnamigen [Moduls](https://docs.python.org/3/library/glob.html).

#### Beispiel

In [ ]:
!ls /DATA/GT/

In [ ]:
import glob

filelist_labels = glob.glob('/DATA/GT/*/*.tif')
filelist_labels

Das Zeichen *\** bedeutet hier eine beliebige Anzahl beliebiger Zeichen. D.h, wir erhalten mit einem Befehl alle tif-Dateien in der zweiten Unterverzeichnissebene unter `DATA/GT/`.

#### Aufgabe 2
- Lasst euch alle tif-Dateipfade der Inputdaten anzeigen.

In [ ]:
#### Lösung

### Einfache Pfadoperationen

Mit der Ausgabe von Dateilisten haben wir noch kein Trainingspaar erstellt. Dazu müssen wir für jeden Eintrag in der Liste der Labeldatenpfade den zugehörigen Pfad in der Liste der Eingabedatenpfade finden. Dabei müssen gleiche Indizes zu zusammengehörigen Datensätze gehören.

Operationen zur Modifikation von Verzeichnissnamen befinden sich im Modul [`os.path`](https://docs.python.org/3.7/library/os.path.html) der Python Standard Library. Wir benötigen in diesem Versuch:
- `basename` - extrahiert einen Dateiname (bzw. Namen des letzten Unterverzeichnisses, wenn ein Verzeichniss angegeben ist).
- `dirname` - gibt den Namen des Elternverzeichnis des gegebenen Pfades zurück.
- `isfile` - überprüft ob eine Datei existiert und gibt *True* oder *False* zurück.
- `join` - fügt mehrere Zeichenketten zu einen Pfad zusammen.

#### Aufgabe 3

1. Erstellt euch ein einfaches Skript um zwei Dateilisten zu erstellen.
    - `x_filelist` beinhaltet alle Inputdatenpfade.
    - `y_filelist` beinhaltet alle zugehörigen Labeldatenpfade.
2. Lasst euch die Anzahl aller vollständigen Input-/Labelpaare ausgeben.  


Hinweis:
- Achtet darauf, dass *Experiment Name* und *Dateiname* in den beiden Pfadlisten für identische Indizes ebenfalls identisch sind.

In [ ]:
#### Lösung

### Bilderausschnitte extrahieren

Wir wollen Netzwerke erstellen, die (wie das  Beispielnetzwerk `simple_model`), mit Inputdateien der Größe $64 \times 64 \times 1$ Pixel arbeiten. Die auf der Festplatte gespeicherten tif-Dateien sind aber übereinander angeordnete 2D Mikroskopaufnahmen. Folglich sind die eingelesenen Arrays 3-dimensional. Der vom Netzwerk verarbeitet Input muss 4D sein.

Dabei definiert die erste Dimension das Bild welches wir betratchten, die zweite Dimension ist die $y$-Postion im Bild, die dritte Dimension die $x$-Position im Bild und die letzte Dimension den Pixelwert für einen bestimmten Kanal.

Die höhere Anzahl an Pixel in den auf der Festplatte gespeicherten Volumen ist kein Nachteil. Im Gegenteil: Sie eröffnen die Möglichkeit, dass wir durch die Auswahl von zufälligen $x$, $y$ und $z$-Ausschnitt) innerhalb des Volumens die Entropie unserer Trainingsdaten weiter erhöhen können und damit gelernte Modell bei der Anwendung robuster sein werden. (In anderen Worten: Der Raum der Inputdaten auf den das Modell gute Vorhersagen ermitteln kann wird größer.)

Bleibt die Frage wie wir Bilder aus den Volumen ausschneiden können. Dazu kann [Indexing](https://docs.scipy.org/doc/numpy-1.10.0/user/basics.indexing.html#other-indexing-options) von Numpy Arrays benutzt werden.

Aufgrund der benutzten Aktivierungsfunktion ist es vorteilhaft die Inputdaten auf einen bestimmten Wertebereich zu normieren. In der Praxis hat sich *zero mean unit variance* (deutsch: Mittelwert von 0 und Varianz von 1) als Wertebereich bewährt.

#### Aufgabe 4
1. Erstellt zwei große Arrays `X` und `Y`
    - `X` beinhaltet Inputbilder mit der Form $64 \times 64 \times 1$.
    - `Y` beinhaltet die den Inputbildern zugeordneten Labelbilder.
   Dazu:
        - [Iteriert](https://docs.python.org/3/tutorial/controlflow.html#for-statements) über alle verfügbare Datenpaare in den Dateilisten.
        - Schneidet an [zufälligen](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.randint.html) Positionen in $x$, $y$ und $z$ Bilder mit der $x$, $y$ Form von $64 \times 64$ Pixel aus.
        - Normiert jedes ausgeschnitte **Inputbild** auf einen [Mittelwert](https://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html#numpy.mean) von 0 und einer [Standardabweichung](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.std.html) von 1 (= Varianz von 1).
        - [Formt](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html) die Bilder in numpy Arrays der Form $64 \times 64 \times 1$ um und speichert sie in `X` in `Y`.
    
2. Nutzt die unten definierte Funktion `plotBatches` um euch die ersten 10 Bilder von `X` und `Y` anzeigen zu lassen.

Hinweis: 
- Die Dimensionen sind umgekehrt gereiht: $(z, y, x)$.
- Nutzt den numpy [Befehl]((https://docs.scipy.org/doc/numpy/reference/constants.html#numpy.newaxis)) `newaxis`.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np

def plotBatches(noisy, gt):
    """Entpackt eine Sammlung von Bildern in mehrere imshow Diagramme."""
    
    f, axes = plt.subplots(noisy.shape[0], 2, figsize=(9.5, noisy.shape[0]*4.5))
    for i in range(noisy.shape[0]):
        (ax1, ax2) = axes[i, :]
        ax1.imshow(np.squeeze(noisy[i, :]))
        ax2.imshow(np.squeeze(gt[i, :]))
    plt.tight_layout()

In [ ]:
#### Lösung

# Schritt für Schritt zum Autoencoder

Im Folgenden wollen wir ein Netzwerk mit ungefähr 98% Genauigkeit erstellen.

Dazu testen wir 5 verschiedene Architekturen. Jede Architektur unterscheidet sich durch die Anzahl der `Pooling` und `UpSampling2D` Schichten. Unten ist eines dieser Netzwerke (`autoencoder1`) mit jeweils einer Pooling und UpSampling Schicht bereits vorgegeben. Beachtet hierbei dass die Funktionen `pooling` und `upsampling` mehrere Netzwerkschichten beinhalten.

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Concatenate, UpSampling2D

def pooling(inputlayer, num_filter):
    pool = MaxPooling2D(pool_size=(2,2))(inputlayer)
    conv = Conv2D(num_filter, 3, activation='relu', padding='same')(pool)
    output = Conv2D(num_filter, 3, activation='relu', padding='same')(conv)
        
    return output

def upsampling(inputlayer, mergelayer, num_filter):
    up = UpSampling2D(size=(2,2))(inputlayer)
    conv1 = Conv2D(num_filter, 2, activation='relu', padding='same')(up)
    merge = Concatenate(axis=3)([mergelayer, conv1])
    conv2 = Conv2D(num_filter, 3, activation='relu', padding='same')(merge)
    output = Conv2D(num_filter, 3, activation='relu', padding='same')(conv2)

    return output

In [ ]:
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

def autoencoder1(input_shape=(64, 64, 1)):
    
    inputs = Input(input_shape)
    num_filter = 64
    
    conv1 = Conv2D(num_filter, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(num_filter, 3, activation='relu', padding='same')(conv1)
    
    pool1 = pooling(conv1, num_filter*2)
    
    up1 = upsampling(pool1, conv1, num_filter)

    outputs = Conv2D(2, 3, activation='relu', padding='same')(up1)
    outputs = Conv2D(1, 1, activation='sigmoid')(outputs)
    
    return Model(inputs=inputs, outputs=outputs)

Gemäß der obigen Definition ist `autoencoder1` wie folgt aufgebaut:
1. `Input`
2. 2x `Conv2D` (64 Kanäle)
3. `pooling` (128 Kanäle) 
4. `upsampling` (64 Kanäle, `Concatenate` mit Ergebnis von Schicht in 2.) )
5. 2x `Conv2D` (Ausgabeschichten)
    
Wir wollen weitere Netzwerke mit folgenden Architekturen aufbauen:

##### autoencoder0
(entspricht `autoencoder1` ohne `pooling` und `upsampling`):
1. `Input`
2. 2x `Conv2D` (64 Kanäle)
4. 2x `Conv2D` (64 Kanäle)
8. 2x `Conv2D` (Ausgabeschichten)
    
##### autoencoder2
(entspricht `autoencoder1` mit jeweils ein zusätzlichen `pooling` und `upsampling` Schritt)
1. `Input`
2. 2x `Conv2D`
3. `pooling` (128 Kanäle) 
4. `pooling` (256 Kanäle) 
5. `upsampling`(128 Kanäle, `Concatenate` mit Ergebnis von Schicht in 3.)
6. `upsampling`(64 Kanäle, `Concatenate` mit Ergebnis von Schicht in 2.)
7. 2x `Conv2D` (Ausgabeschichten)

##### autoencoder3
(entspricht `autoencoder1` mit jeweils zwei zusätzlichen `pooling` und `upsampling` Schritten)
1. `Input`
2. 2x `Conv2D`
3. `pooling` (128 Kanäle) 
4. `pooling` (256 Kanäle) 
5. `pooling` (512 Kanäle)
6. `upsampling` (256 Kanäle, `Concatenate` mit Ergebnis von Schicht in 4.)
7. `upsampling` (128 Kanäle, `Concatenate` mit Ergebnis von Schicht in 3.)
8. `upsampling` (64 Kanäle, `Concatenate` mit Ergebnis von Schicht in 2.)
9. 2x `Conv2D` (Ausgabeschichten)


##### autoencoder4
(entspricht `autoencoder1` mit jeweils drei zusätzlichen `pooling` und `upsampling` Schritten)
1. `Input`
2. 2x `Conv2D`
3. `pooling` (128 Kanäle) 
4. `pooling` (256 Kanäle) 
5. `pooling` (512 Kanäle)
6. `pooling` (1024 Kanäle)
8. `upsampling` (512 Kanäle, `Concatenate` mit Ergebnis von Schicht in 5.)
9. `upsampling` ( 256 Kanäle, `Concatenate` mit Ergebnis von Schicht in 4.)
10. `upsampling` (128 Kanäle, `Concatenate` mit Ergebnis von Schicht in 3.)
11. `upsampling` (64 Kanäle, `Concatenate` mit Ergebnis von Schicht in 2.)
12. 2x `Conv2D` (Ausgabeschichten)

Dabei ist zu beachten, dass die Ausgabeschichten, die Inputschicht und die ersten beiden Konvolutionen unverändert bleiben.

#### Aufgabe 5
- Erstellt Funktionen für die Netzwerke `autoencoder0`, `autoencoder2`, `autoencoder3` und `autoencoder4`.
- <a href="#Beispiel:-Kompilieren-eines-Modells">Kompiliert</a> die Modelle mit
    - der Adam Optimierungsfunktion mit einer Anpassungsrate von `1e-4`
    - der Verlustfunktion `binary_crossentropy'
    - der Metrik `accuracy`
- <a href="#Beispiel:-Trainieren-eines-Modells">Trainiert</a> die Modelle mit
    - `X`, `Y` als Input- bzw. Labeldaten
    - einem Validationsanteil von 10%
    - für 100 Epochen
- <a href="#Beispiel:-Netzwerk-und-Trainingsverlauf-für-die-spätere-Auswertung/-Anwendung-speichern">Speichert</a> die trainierten Modelle auf der Festplatte.

In [ ]:
#### Lösung

In <a href="#Aufgabe-5">Aufgabe 5</a> wurden eine ganze Reihe von Netzwerken erstellt und trainiert. Für den weiteren Verlauf des Praktikums wollen wir nun das beste Netzwerk auswählen. Die Frage ist nun: Welches kann am besten mit unbekannte Mikroskopdaten umgehen. Die Daten der <a href="#Aufgabe-5">gespeicherten</a> Trainingsläufe könnt ihr mit der unten definierten Funktion `readHistory` von der Festplatte lesen.

In [ ]:
import numpy as np
import glob
import os

def readHistory(foldername, latest=True):
    history_files = glob.glob(os.path.join(foldername, '*', 'history.csv'))
    
    if latest:
        dates = np.asarray([os.stat(history_file).st_ctime for history_file in history_files])
        idx = np.argsort(dates)[-1]
        data = np.genfromtxt(history_files[idx], delimiter=',', skip_header=1)
        header = np.genfromtxt(history_files[idx], delimiter=',', max_rows=1, dtype=str)
        
        return data, header
    
    else:
        data = [np.genfromtxt(filename, delimiter=',', skip_header=1) for filename in history_files]
        header = [np.genfromtxt(filename, delimiter=',', max_rows=1, dtype=str) for filename in history_files]
        
        return data, header

#### Aufgabe 6
- <a href="#Beispiel:-Visualisieren-eines-Trainingsprozesses">Erstellt</a> zwei Diagramme:
    1. Verlauf der Trainingsgenauigkeit über die Trainingsepochen.
    2. Verlauf der Validierungsgenauigkeit über die Trainingsepochen.
- Ladet mit der Funktion `load_trained_model` das Modell mit der besten Validierungsgenauigkeit in das aktuelle Notebook.

In [ ]:
#### Lösung

# Treffe Voraussagen auf noch unbekannte Daten

Im zweiten Teil des Praktikums wollen wir uns auf eine wesentliche Fragestellung in unserer Arbeitsgruppe konzentrieren:

- Welche physikalischen Zelleigenschaften können wir aus noch unbekannten Mikroskopaufnahmen extrahieren?

Dazu sollt ihr **eine** Mikroskopaufnahmen aus `/DATA/validation/noise1000/` auswählen und im weiteren Verlauf verwenden.

In [ ]:
!ls /DATA/validation/noise1000/

## Isotropische Auflösung

Um wirklich quantitative Aussagen über Zellen zu treffen (Volumen, Intensität, Abstände), müssen wir beachten, dass
durch das Mikroskopsetup in unterschiedliche Raumrichtungen unterschiedliche Auflösungen existieren. In den Beispielbildern wird von einem Pixelabstand von $63,2\,\mathrm{nm}$ in $x$ und $y$-Richtung und von $400\,\mathrm{nm}$ in $z$-Richtung ausgegangen.

Für isotropische Auflösungen wollen müssen wir den Pixelabstand in $z$ Richtung auf ebenfalls $63,2\,\mathrm{nm}$ linear interpolieren.

Dazu bietet sich die [Funktion](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.affine_transform.html) `affine_transform` im Modul `scipy.ndimage` an. Die wesentlichen benötigten Parameter sind:
 - Das Usprungsvolumen.
 - Die Transformationsmatrix (eine $3 \times 3$ [Diagonalmatrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.diag.html#numpy.diag), die auf der Hauptdiagonale die Einträg $\left(d, 1, 1\right)$ besitzt, wobei $d$ der inverse Streckungsfaktor in $z$-Richtung darstellt).
 - `output_shape` muss angegeben werden. Dieser ist bis auf die Anzahl an $z$-Ebenen identisch mit der ursprünglichen Array-Form.
 - `order=1` welches die Interpolation auf die lineare Ordnung beschränkt.

#### Aufgabe 7
- Berechnet die lineare Interpolation von einer ausgewählten .tif Dateien in `/DATA/validation/`
- [Speichert](https://docs.scipy.org/doc/numpy/reference/generated/numpy.save.html) die Interpolation als `.npy`-Datei auf der Festplatte.
- Erstellt eine [Maximumsprojektion](https://docs.scipy.org/doc/numpy/reference/generated/numpy.amax.html#numpy.amax) aller 3 Raumachsen.

In [ ]:
#### Lösung

Das trainierte Netzwerk geht von an zufälligen Positionen im Volumen extrahierten 2D Minibatches mit der Größe von $64 \times 64$ Pixel aus.

Dies kann nicht für die Vorhersage des kompletten Volumens benutzt werden. Wir müssen das Volumen in kleine Teilbilder zerschneiden. Dabei ist es wichtig, dass eine festgelegte Reihenfolge eingehalten wird, um hinterher die Vorhersagen wieder zu einem kompletten Volumen zusammensetzen zu können.

#### Aufgabe 8
- Nutzt das trainierte Modell und `predictImageStack` um in den interpolierten Volumen die Zellen zu erkennen.
- Speichert als `.npy`-Datei`:
    - die Zellvorhersagen
    - die Intensitätsbilder
- Berechnet von jeder Zellvorhersage eine [Durchschnittsprojektion](https://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html) in $x$, $y$ und $z$-Richtung.
- Erstellt ein Histogramm der Pixelwerte in der Vorhersage

Hinweis: Benutzt für das Histogram [logarithmische Auftragung](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.set_yscale.html) auf der y-Achse und mindestens 100 Abschnitte.

In [ ]:
def predictImageStack(img, model, num_channels=1, num_classes=1, batch_size=32):
    print('Image shape: {}'.format(img.shape))
    input_shape = model.layers[0].output_shape
    print('Model input shape: {}'.format(input_shape))
    
    in_z, in_y, in_x = img.shape
    height, width = input_shape[1:3]
    nz = in_z - 1
    ny = int(np.floor(in_y / height))
    nx = int(np.floor(in_x / width))
    
    print('Number of tiles in x, y, z: ({:d}, {:d}, {:d})'.format(nx, ny, nz))
    
    margin_x = (in_x - (width * nx)) / 2
    margin_y = (in_y - (height * ny)) / 2
    
    
    img = img[..., None]
    imgs = ()
    for _ in range(num_channels):
        imgs = imgs + (img,)
    img = np.concatenate(imgs, axis=-1)
    
    if margin_x == 0.0:
        x_start = None
        x_end = None
        
    else:
        x_start = int(np.floor(margin_x))
        x_end = -int(np.ceil(margin_x))
    
    if margin_y == 0.0:
        y_start = None
        y_end = None
    else:
        y_start = int(np.floor(margin_y))
        y_end = -int(np.ceil(margin_y))
        
    # cut away overview plane in z and reduce size in x, y to fit model input
    img_ = img[:, y_start:y_end, x_start:x_end, :]
    
    print('cut image to shape: {}'.format(img_.shape))
    
    num_batches = int(nx * ny * nz)
    
    print('Total number of tiles: {}'.format(num_batches))
    
    prediction_batches = np.zeros((num_batches, height, width, num_channels))
    
    print('Prediction batches shape: {}'.format(prediction_batches.shape))
    
    def norm(a, axis=None):
        a = a - np.mean(a, axis=axis, keepdims=True)
        return a / np.std(a, axis=axis, keepdims=True)
    
    
    print('Fill input batches with image data')
    k = 0
    for i in range(ny):
        for j in range(nx):
            for l in range(nz):
                prediction_batches[k, :, :, :] = norm(img_[l, height*i:height*(i+1), width*j:width*(j+1)], axis=(0, 1, 2))
                k += 1
                
    num_predictions = num_batches // batch_size
    
    print('Predict batches')
    for i in range(num_predictions):
        prediction_batches[i*batch_size:(i+1)*batch_size] = model.predict(prediction_batches[i*batch_size:(i+1)*batch_size])
    
    if num_predictions*batch_size != num_batches:
        prediction_batches[num_predictions*batch_size:] = model.predict(prediction_batches[num_predictions*batch_size:])
    
    img_prediction = np.zeros((*img_.shape[:3], num_classes))
    
    print('Constuct predicted image with shape: {}'.format(img_prediction.shape))
    
    k = 0
    for i in range(ny):
        for j in range(nx):
            for l in range(nz):
                img_prediction[l, height*i:height*(i+1), width*j:width*(j+1)] = prediction_batches[k, ..., :num_classes]
                k += 1

    
    return np.squeeze(img_prediction), np.squeeze(img_)


In [ ]:
#### Lösung

# Auswertung

**Die Auswertung ist mit den gespeicherten Werten auch Zuhause oder im Lernzentrum möglich.**


Mit den Vorhersagen wollen wir jetzt weiterarbeiten. Momentan stehen wir noch vor einigen Problemen:
1. Die Vorhersage muss diskretisiert werden (von \[0, 1\] zu \{0, 1\})
2. Wir wollen Objekte, die nicht miteinander verbunden sind mit unterschiedlichen [Label](http://scikit-image.org/docs/dev/api/skimage.measure.html#label) versehen.

#### Aufgabe 9:
- Erstellt eine Liste, die für jede abgespeicherte Vorhersage die zugehörige Labelmatrix enthält. Dazu:
    - Vergleicht jedes Pixel mit einem von euch definierten Grenzwert
    - Erstellt die Labelmatrix auf Grundlage des Vergleichs.

Hinweis: 
- Die [Vergleichsoperatoren](https://www.tutorialspoint.com/python/python_basic_operators.htm) können nicht nur Skalarwerte miteinander vergleichen, sondern funktionieren auch beim Vergleich von numpy Arrays mit Skalarwerten. Das Resultat ist dann eine Matrix aus *True* und *False* entsprechend der Vergleichsoperation.

In [ ]:
#### Lösung

Aus diesen segmentierten Daten kann man schon viele Eigenschaften berechnen. Im Detail sind dies:
- Zellpositionen (Beispiel)
- Mittlere/ Maximale/ Minimale  Helligkeit 
- Zellvolumen
- Lokale Dichte (mit bereitgestellten Funktion)
- Zellabstände (optional)
- Nemantic Orderparamter (optional)

## Zelleigenschaften bestimmen

#### Beispiel

Sei `w` eine oben bestimmte Labelmatrix (bzw. das Labelvolumen) und `img_` die der Segmentierung zugehörige Intensitätswerte, dann lässt sich ein numpy Array `centroids_array` mit allen Schwerpunkten aller Objekten in der Labelmatrix bestimmen via:
```python
from skimage.measure import regionprops

props = regionprops(w, img_)
centroids_list = [props[i].centroid for i in range(len(props))]
centroids_array = np.asarray(centroids_list)
```

Dabei beinhaltet das Ergebnis von `skimage.measure.regionprops` noch eine ganze Reihe von weiteren [Objekteigenschaften](http://scikit-image.org/docs/dev/api/skimage.measure.html#regionprops).

#### Aufgabe 10
- Bestimmt folgende für jedes gelabelte Volumen
    - alle Zellpositionen
    - alle mittlere, maximale und minimale Helligkeit der gelabelten Zellen
    - alle Zellvolumen
    
- Erstellt für jedes der Labelvolumen und jeden (skalarwertigen) Parameter ein entsprechendes Histogram.

Hinweis:
- Der für das Volumen benötigte Objekteingenschaft heißt *area*, da usprünglich nur 2D Eigenschaften extrahiert werden konnten.

In [ ]:
#### Lösung

## Lokale Dichte

Obwohl Position, Helligkeit und Volumen interessante Zellparameter sind, können wir aus den Segmentierung noch wesentlich mehr Informationen gewinnen. In den meisten Veröffentlichungen konzentiert man sich auf einige wenige Eigenschaften. Wirklich relevante Wissenschaft produziert man wenn nur, wenn man die ausgetretenen Pfade verlässt. Dazu muss man sich überlegen wie man Messwerte aus den Bildern ermitteln kann, für die noch keine vorgefertigte Funktion existiert.

Ein solches Beispiel wäre die lokale Dichte. Eine Beispielimplementierung findet ihr in der Funktion `calculateLocalDensity`.

In [ ]:
from numpy.linalg import norm

def calculateLocalDensity(w, centroid_array, radius = 60):
    """ Calculates occupied volume in a sphere around centroid
    """

    # Erstellt ein Koordinatengitter
    Z, Y, X = np.meshgrid(np.arange(-radius, radius, 1), np.arange(-radius, radius, 1), np.arange(-radius, radius, 1))
    
    # Wandelt das 3D Gitter in eine Sammlung der Gitterpunktkoordinaten um
    Z, Y, X = np.ravel(Z), np.ravel(Y), np.ravel(X)
    
    # Bestimmt für jede Gitterpunktkoordinate ob sie Teil der Kugel ist oder nicht.
    sphere = norm([Z, Y, X], axis=0) < radius
    
    # Das (einheitenlose) Volumen der Kugel ist die Summe alle Kugelvoxel
    vol_sphere = np.sum(sphere) 

    # Für die lokale Dichte wollen wir jeden Zellvoxel (unabhängig von dem Labelwert) mit 1 bezeichnen
    w_ = w > 0
    
    # Damit wir nicht durch Randeffekte das Ergebnis verfälschen,
    # ergänzen wir an den Volumenrändern Voxel mit dem Wert -1
    w_ = np.pad(w_, radius, 'constant', constant_values=-1)

    # Erzeugen den Ausgabevektor
    localDensity = np.zeros(centroid_array.shape[0])
    
    # Iterieren über Zellpositionen
    for i in range(centroid_array.shape[0]):
        z, y, x = centroid_array[i]
        z, y, x = np.round([z, y, x])
        
        # Schneiden um jede Zellvolumen einen Würfel mit der Kantenlänge 2*radius aus
        w_part = w_[int(z):int(z+2*radius), int(y):int(y+2*radius), int(x):int(x+2*radius)]
        
        # Nur Beiträge die Teil der Kugel sind sollen zur Berechnung der lokalen Dichte beitragen 
        vol = w_part.flatten()*sphere
        
        # Beiträge die mit -1 versehen sind sind außerhalb der Labelmatrix und sollen nicht zum Volumen
        out = np.sum(vol == -1)
        
        # Berechnung der lokalen Dichte
        localDensity[i] = np.sum(vol==1)/(vol_sphere - out);

    return localDensity

# Visualisierung


Durch die Mikroskopie kennen wir nicht nur den Wert von Zelleigenschaften, sondern auch die räumliche Verteilung. Um diese zu Visualisieren eignen sich u.A. 3D Scatterplots.

In der *matplotlib* [Dokumentation](https://matplotlib.org/examples/mplot3d/scatter3d_demo.html) findet ihr ein Beispiel dazu. Wir haben uns erlaubt diese Beispiel weiter zu vereinfachen:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

n = 100

x = np.random.randn(n)
y = np.random.randn(n)
z = np.random.randn(n)

fig = plt.figure(figsize=(9.5, 9))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x, y, z, c='r', marker='o')

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')
ax.set_title('Beispiel 3D Scatter Plot')

plt.tight_layout()

#### Aufgabe 11
- Erstellt für die 3 Labelvolumen ein 3D Scatterplot für die durchschnittliche Helligkeit und die lokale Dichte.
    - Die Markerposition soll durch den jeweiligen Schwerpunkt gegeben sein.
    - Die Markerfarbe soll den Parameter darstellen.
    
Hinweis:
- Berechnet die lokale Dichte mit der gegebenen Funktion `calculateLocalDensity`.
- Ihr könnt über die Farbe der Punkte (`c=`) die Intensität oder die Dichte kodieren.
- die `x, y, z` erhaltet ihr über die oben ermittelten Schwerpunkte

In [ ]:
#### Lösung

# 3D Rendering (optional)

Zum Abschluss stehen euch zwei Möglichkeiten offen 3D Visualisierungen der Zellen zu erstellen.

## *matplotlib* (nicht empfohlen)

Kann in diesem Notebook gemacht werden, dauert aber lange und die Plots lassen sich nicht wirklich interativ drehen.

**Bitte entfernt die Anführungszeichen nur, wenn ihr bereit seit sehr lange auf das Ergebnis zu warten!**

In [ ]:
from skimage.measure import marching_cubes_lewiner

verts, faces, normals, values = marching_cubes_lewiner(dilate, 0.5)

"""
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(verts[:, 0], verts[:,1], faces, verts[:, 2],
                cmap='Spectral', lw=1)

plt.show()
"""

## *ParaView* (empfohlen)

ParaView  ist ein 3D Rendering Programm für große Datensätze und de facto Standard für die Visualisierung von aufwendigen Simulation. Die folgende Handreichung soll euch dazu ermuntern ParaView zum Rendern einer 3D Darstellung zu benutzen. (beispielsweise kann man mit ParaView Abbildungen wie z.B. in der [Versuchsbeschreibung](https://www.uni-marburg.de/de/fb13/studium/praktika/praktika-fuer-physiker/v-so-15-maschinelles-lernen) erstellen oder gar ganze Simulationsreihen in einen [Film](https://static-content.springer.com/esm/art%3A10.1038%2Fs41567-018-0356-9/MediaObjects/41567_2018_356_MOESM3_ESM.mp4) umwandeln).

In [ ]:
# Export to VTI
from pyevtk.hl import imageToVTK

imageToVTK('prediction', pointData={"prediction":w})

## Download

Ladet bitte den ParaView Installer von der offiziellen [Website](https://www.paraview.org/download/) herunter. Im Folgenden wird die Windowsversion benutzt. Für Max OSX und der Linux Distribution eurer Wahl finden sich entsprechende Versionen im Mac App Store und der Paketverwaltung.

## Datentransfer

Um die Daten auf euren lokalen Computer herunterladen zu können, könnt ihr die jupyter [Homepage](http://localhost:8888/)  benutzen. Wesentlich komfortabler geht es aber über ein Programm, welches Daten über eine [SSH](https://de.wikipedia.org/wiki/Secure_Shell)-Verbindung übertragen kann. Unter Mac OSX und Linux könnt ihr euren normalen Dateimanager benutzen. Unter Windows benötigt ihr ein zusätzliches Programm (z.B. [WinSCP](https://winscp.net/eng/download.php)).

Für eine Verbindung mit WinSCP benötigt Ihr folgende Daten:
- File protocol: SFTP
- Host name: ***
- User name: ***
- Passwort: ***

Unter nautilus (Linux Dateimanager) findet ihr das entsprechende Menü unter 'Other Locations' und 'Connect to Server'. In die Addresszeile müsst ihr eintragen:
`sftp://<username>@<hostname>/`

Bitte ladet die `.vti`-Dateien unter `/home/<username>/Notebooks` herunter

## Datenvisualisierung in ParaView

* Öffnet die Datei.
* Nutzt den Threshold Filter im Bereich eurer Labels.
* Unter Coloring `prediction` auswählen.
* Spielt ein bisschen mit den Einstellungen im Bereich *OSPRay Rendering* bis ihr eine schöne 3D Visualizierung erstellt habt.

Hinweise: 
* Für Schatten benötigt man eine Fläche die Schatten abbilden.
* *path tracer* ist wesentlich schöner anzusehen, kostet aber auch sehr viel Rechenleistung.

# Abschluss

1. Speichert ALLE Daten/ Notebooks/ DIAGRAMME
2. Ladet ALLE .ipynb, .npy, .vti herunter
3. Ladet alle .tif Bilder herunter


# Sonstiges

- Wenn Ihr weiter mit Deep Learning/ Python Notebooks experimentieren möchtet, aber keinen Computer mit GPU zur Verfügung habt:<br></br>
https://colab.research.google.com/notebooks/welcome.ipynb (Account wird benötigt)

- Der Jupyter Notebook Server ist Teil der Anaconda Distribution:<br></br>
https://www.anaconda.com/distribution/ (ist im Lernzentrum installiert)

- Wenn ihr eine leistungsstarken (Linux) Computer zur Verfügung habt und euch keine Arbeit mit dem Erstellen einer jupyter/ tensorflow/ GPU-Umgebung machen wollt:<br></br>
https://www.tensorflow.org/install/docker (Für das Praktikum verwendete tag: 1.13.1-gpu-py3-jupyter)

- Die aktuelle Version dieses Notebooks findet ihr sowohl im Fachbereichs [Git-Repository](https://git.physik.uni-marburg.de/Jelli/f_praktikum_sose_v15) (Intranet) und auf [Github](https://github.com/erjel/F-Prak-SoSe-V15). Verbesserungsvorschläge sind immer willkommen!